In [1]:
import networkx as nx
import numpy
import igl
from meshplot import plot

# add edges to graph G for each adjacent face in mesh
def add_edges_from_mesh_igl(G,f):
    m,n = igl.triangle_triangle_adjacency(f)
    for face_id, face in enumerate(f):
        for j, ajd_face_id in enumerate(m[face_id]):
            G.add_edge(face_id,ajd_face_id)
# adds nodes to graph G for every face in mesh
def add_nodes_from_mesh_igl(G,f): [G.add_node(face_id) for face_id, face in enumerate(f)]
# create networkx graph from given mesh
def graph_from_mesh_igl(v,f):
    G = nx.Graph()
    add_nodes_from_mesh_igl(G,f)
    add_edges_from_mesh_igl(G,f)
    return G
# create spanning tree from mesh m using networkx
def spanning_tree_from_mesh_igl(v,f): return nx.minimum_spanning_tree(graph_from_mesh_igl(v,f))
# returns faces adjacent to given face
def find_adjacent_faces (face_id,face_adjacency):
    adjacent_faces = face_adjacency[face_id]
    return adjacent_faces
# returns the adjacent nodes to node n in tree T
def get_adjacent_nodes(T,n): return nx.descendants_at_distance(T,n,1)
# returns list of triangulated faces in the current node (orignal face)
def get_faces_in_current_node(mapping, current_node): return mapping[current_node]
# returns list of triangulated faces in the nodes adjacent to the current node (orignal face)
def get_faces_in_adjacent_node(spanning_tree, mapping, current_node): 
    adjacent_nodes = get_adjacent_nodes(spanning_tree,current_node)
    faces_in_adjacent_node = []
    for i, node in enumerate (adjacent_nodes):
        faces_in_adjacent_node = numpy.append(faces_in_adjacent_node, mapping[node])
    return faces_in_adjacent_node,adjacent_nodes
# returns next triangulated face to visit and new node if shiting to adjacent node
def pick_next_face_from_adjacent_node (current_list_of_faces, face_id, current_node, possible_next_faces, faces_in_adjacent_node, adjacent_nodes, mapping):
    for adjacent_face in possible_next_faces:
        if adjacent_face not in current_list_of_faces and adjacent_face in faces_in_adjacent_node:
            face_id = adjacent_face
            for i, node in enumerate (adjacent_nodes):
                if face_id in mapping[node]:
                    current_node=node
    return face_id, current_node
# returns next triangulated face to visit and current node if staying in current node
def pick_next_face_from_current_node (current_list_of_faces, face_id, possible_next_faces, faces_in_current_node):
    for adjacent_face in possible_next_faces:
        if adjacent_face not in current_list_of_faces and adjacent_face in faces_in_current_node:
            face_id = adjacent_face
    return face_id
# returns the midpoints of the edges of a triangle t
def calc_edge_midpoints(t): return (t[0] + t[1])/2, (t[1] + t[2])/2, (t[2] + t[0])/2
# return midpoints of all triangles in a given mesh
def calc_triangle_centers(vertices, faces): return igl.barycenter(vertices, faces)



# triangulates the mesh and returns array of new vertex coordinates and faces
def triangulated_mesh_igl(original_vertices, original_faces):
    #find the center vertex coordinate of each orignal face
    center_vertices = calc_triangle_centers(original_vertices, original_faces)
    #initialize new vertex coordinate arrays
    triangulated_vertices = numpy.array([original_vertices[0]])
    triangulated_faces = numpy.array([original_faces[0]])
    test_triangulated_faces = numpy.array([original_faces[0]])
    #add original vertex coordinates, center and midpoint vertices to new vertex coordinate array
    #maps between orignal_faces and new triangulated_faces
    dict = {}
    for face_id, face in enumerate(original_faces):
        # create the six triangulated faces (vertices in clockwise order)
        new_faces = [
                    [0, 1, 2],
                    [0, 2, 3],
                    [0, 3, 4],
                    [0, 4, 5],
                    [0, 5, 6],
                    [0, 6, 1],
                            ]
        # create map and offset the face ids by the existing faces
        dict[face_id] = numpy.array([0, 1, 2, 3, 4, 5]) + len(triangulated_faces)-1
        # offset the vertice ids by the existing vertices
        triangulated_faces = numpy.append(triangulated_faces, numpy.array(new_faces) + len(triangulated_vertices), axis=0)
        #calculate the midpoints of the edges and include them along with the old vertices and center points
        midpoint_vertices = calc_edge_midpoints([original_vertices[face[0]], original_vertices[face[1]], original_vertices[face[2]]])
        triangulated_vertices = numpy.append(triangulated_vertices,  
                                                  [center_vertices[face_id],
                                                   original_vertices[face[0]],
                                                   midpoint_vertices [0],
                                                   original_vertices[face[1]],
                                                   midpoint_vertices [1],
                                                   original_vertices[face[2]],
                                                   midpoint_vertices [2]],axis=0)
    return triangulated_vertices, triangulated_faces, dict

# traversal function that generates list of triangulated faces in order of walk around spanning tree of orignal faces
def traverse_igl(spanning_tree,clean_triangulated_faces,mapping): 
    face_adjacency,n = igl.triangle_triangle_adjacency(clean_triangulated_faces)
    face_id=0
    tree_node_id=0
    list_of_faces = [0]
    for i in range(len(clean_triangulated_faces)):
        list_of_faces=numpy.append(list_of_faces,face_id)
        possible_next_faces = find_adjacent_faces(face_id,face_adjacency)
        faces_in_current_node = get_faces_in_current_node(mapping,tree_node_id)
        faces_in_adjacent_node,adjacent_nodes = get_faces_in_adjacent_node(spanning_tree,mapping,tree_node_id)
        face_id_new, tree_node_id = pick_next_face_from_adjacent_node(list_of_faces, face_id, tree_node_id, possible_next_faces, faces_in_adjacent_node, adjacent_nodes, mapping)
        if face_id_new == face_id:
            face_id_new = pick_next_face_from_current_node(list_of_faces, face_id, possible_next_faces, faces_in_current_node)
        face_id = face_id_new

    #check if all the vertices are visited
    #print(len(list_of_faces))
    #make sure none of the vertices were visited twice
    #for i in range(0, len(list_of_faces)):    
    #    for j in range(i+1, len(list_of_faces)):    
    #        if(list_of_faces[i] == list_of_faces[j]):    
    #            print(list_of_faces[j]);
    return list_of_faces [1:]

def hamiltonian_refinement_igl(orignal_vertices,orignal_faces):
    #triangulate the mesh
    triangulated_vertices,triangulated_faces,mapping = triangulated_mesh_igl(orignal_vertices,orignal_faces)
    #clean the triangulate the mesh, removing duplicated vertices created during triangulation
    clean_triangulated_vertices, clean_triangulated_faces, h = igl.remove_duplicates (triangulated_vertices,triangulated_faces,0.00001)
    #create spanning tree from the orignal mesh
    Tree = spanning_tree_from_mesh_igl(orignal_vertices, orignal_faces)
    #traverse the triangulated mesh along the spanning tree route
    list_of_faces = traverse_igl(Tree,clean_triangulated_faces,mapping)
    #TODO map faces to triangles so strip folding can be applied
    face_coordinate_list=[]
    for i in range (0, len(list_of_faces)):
        face = clean_triangulated_faces[list_of_faces[i]]
        face_coordinate_list.append([clean_triangulated_vertices[vertex_id] for vertex_id in face])
    return face_coordinate_list

v, f = igl.read_triangle_mesh("cube.stl")
#clean up all duplicate vertices from .stl mesh
#removing duplicates does not alter the faces, the vertex list is cleaned up but the face ids remain the same and point to the same vertices (but at different vertex ids)
clean_v, clean_f, h = igl.remove_duplicates (v,f,0.00001)
face_coordinate_list = hamiltonian_refinement_igl(clean_v,clean_f)
print (face_coordinate_list)

[[array([5.        , 4.66666667, 6.66666667]), array([ 5., -2., 20.]), array([ 5., -2., 10.])], [array([-1.66666667, -2.        , 13.33333333]), array([ 5., -2., 10.]), array([ 5., -2., 20.])], [array([-1.66666667, -2.        , 13.33333333]), array([ 5., -2., 20.]), array([-5., -2., 20.])], [array([-1.66666667, -2.        , 13.33333333]), array([-5., -2., 20.]), array([-15.,  -2.,  20.])], [array([-1.66666667, -2.        , 13.33333333]), array([-15.,  -2.,  20.]), array([-5., -2., 10.])], [array([-1.66666667, -2.        , 13.33333333]), array([-5., -2., 10.]), array([ 5., -2.,  0.])], [array([-1.66666667, -2.        , 13.33333333]), array([ 5., -2.,  0.]), array([ 5., -2., 10.])], [array([5.        , 4.66666667, 6.66666667]), array([ 5., -2., 10.]), array([ 5., -2.,  0.])], [array([5.        , 4.66666667, 6.66666667]), array([ 5., -2.,  0.]), array([5., 8., 0.])], [array([-1.66666667, 11.33333333,  0.        ]), array([5., 8., 0.]), array([ 5., -2.,  0.])], [array([-1.66666667, 11.3333